<a href="https://colab.research.google.com/github/alexislintw/Movie-Recommender-Systems/blob/main/rs_user_collaborative_filtering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movie Recommender System
### Base on User Collaborative Filtering
- Author: Alexis
- Created: 2021/2/24
- Updated: 2022/2/27


### Load and organize the data

In [ ]:
import pandas as pd

PATH = '/content/drive/MyDrive/Dataset/ml-latest-small/'
ratings = pd.read_csv(PATH + 'ratings.csv')
movies = pd.read_csv(PATH + 'movies.csv')

print(ratings.columns)

# 只取需要的欄位
ratings = ratings[['userId', 'movieId', 'rating']]

# 轉成『用戶-物品』評分矩陣
rating_pivot = ratings.pivot(index="userId",columns="movieId",values="rating").fillna(0)
rating_pivot

Index(['userId', 'movieId', 'rating', 'timestamp'], dtype='object')


movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
606,2.5,0.0,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,2.5,0.0,0.0,0.0,3.5,0.0,4.0,4.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,4.5,0.0,0.0,4.0,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
607,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
608,2.5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,2.0,0.0,3.5,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,3.5,3.5,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
print(rating_pivot.shape)
print(rating_pivot.index)
print(rating_pivot.columns)

(610, 9724)
Int64Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
            ...
            601, 602, 603, 604, 605, 606, 607, 608, 609, 610],
           dtype='int64', name='userId', length=610)
Int64Index([     1,      2,      3,      4,      5,      6,      7,      8,
                 9,     10,
            ...
            193565, 193567, 193571, 193573, 193579, 193581, 193583, 193585,
            193587, 193609],
           dtype='int64', name='movieId', length=9724)


### Generate the matrix of the user similarity

In [ ]:
import numpy as np

# 轉成是否有評分
rating_pivot[rating_pivot > 0] = 1
print(rating_pivot.head())

# 共現矩陣C (評分矩陣與評語矩陣轉置的內積)
#C = A.dot(A.T)
C = rating_pivot@rating_pivot.T
print(C)

### TODO
# 懲罰熱門物品
# count[u][v] += 1 / math.log(1+len(users))
### TODO

# 取出共現矩陣 C 的對角，為每個用戶各自的評分次數
arr1 = C.to_numpy()
user_rating_counts = np.diagonal(arr1)

# 複製一維陣列為矩陣
arr2 = np.tile(user_rating_counts, (len(user_rating_counts),1))
X = pd.DataFrame(data=arr2.T)
print(X)

# 分母部份：C' = N(u)*N(v)
C_ = X*X.T
C_.index = range(1,C_.shape[0]+1)
C_.columns = range(1,C_.shape[0]+1)
print(C_)

# 分母部份：C'開根號
D = np.sqrt(C_)
print(D)

#計算最後的相似度矩陣
S = C/D
S

movieId  1       2       3       4       ...  193583  193585  193587  193609
userId                                   ...                                
1           1.0     0.0     1.0     0.0  ...     0.0     0.0     0.0     0.0
2           0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
3           0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
4           0.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0
5           1.0     0.0     0.0     0.0  ...     0.0     0.0     0.0     0.0

[5 rows x 9724 columns]
userId    1     2     3      4     5    ...     606    607    608   609     610
userId                                  ...                                    
1       232.0   2.0   7.0   45.0  13.0  ...    84.0   60.0  136.0   9.0    69.0
2         2.0  29.0   0.0    1.0   1.0  ...     5.0    1.0    6.0   1.0    18.0
3         7.0   0.0  39.0    1.0   1.0  ...     8.0    4.0    9.0   0.0     7.0
4        45.0   1.0   1.0  216.0  12

userId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.024383,0.073590,0.201021,0.128669,0.122266,0.138455,0.143648,0.048400,0.033292,0.131306,0.023212,0.070750,0.123191,0.141263,0.172431,0.243470,0.205117,0.304567,0.139272,0.146606,0.042129,0.101464,0.156495,0.090130,0.100287,0.259924,0.200743,0.145896,0.090075,0.157841,0.143014,0.141925,0.106194,0.082138,0.084758,0.157594,0.141241,0.334831,0.103504,...,0.086851,0.147655,0.250590,0.095827,0.054711,0.044042,0.320800,0.000000,0.161367,0.242105,0.051903,0.043866,0.052640,0.115302,0.109278,0.122910,0.127777,0.184239,0.124568,0.311459,0.125080,0.135432,0.174663,0.150862,0.102764,0.119822,0.299451,0.028653,0.237398,0.285217,0.071860,0.152564,0.235176,0.078784,0.154571,0.165157,0.288062,0.309738,0.097140,0.125545
2,0.024383,1.000000,0.000000,0.012635,0.027995,0.020959,0.045186,0.027086,0.000000,0.078471,0.046424,0.000000,0.033352,0.026803,0.159821,0.093790,0.108732,0.165760,0.014007,0.011937,0.097049,0.119159,0.016881,0.123937,0.218507,0.000000,0.000000,0.054445,0.103164,0.159232,0.000000,0.018387,0.029735,0.040048,0.000000,0.047946,0.040522,0.021026,0.000000,0.018297,...,0.000000,0.029831,0.171825,0.038720,0.000000,0.000000,0.014635,0.000000,0.000000,0.053359,0.146805,0.223331,0.049629,0.000000,0.118879,0.090130,0.000000,0.049629,0.029361,0.048176,0.025270,0.038306,0.054891,0.012191,0.000000,0.100756,0.008823,0.040522,0.082068,0.020168,0.203251,0.015982,0.012094,0.000000,0.000000,0.027806,0.013579,0.038650,0.030528,0.092633
3,0.073590,0.000000,1.000000,0.010895,0.024140,0.027110,0.000000,0.023357,0.000000,0.000000,0.000000,0.000000,0.000000,0.023113,0.041345,0.048526,0.062508,0.042881,0.030197,0.051467,0.022824,0.029358,0.014557,0.015268,0.031404,0.000000,0.013782,0.060363,0.035584,0.000000,0.022646,0.031710,0.038462,0.017267,0.000000,0.000000,0.000000,0.018131,0.032026,0.015778,...,0.030261,0.051447,0.000000,0.000000,0.000000,0.071611,0.063099,0.000000,0.037483,0.053681,0.050637,0.000000,0.000000,0.000000,0.000000,0.011103,0.037398,0.021398,0.025318,0.065282,0.000000,0.000000,0.047334,0.010513,0.000000,0.039493,0.053255,0.000000,0.064335,0.052173,0.031867,0.027563,0.046930,0.000000,0.010771,0.038364,0.046839,0.049993,0.000000,0.031064
4,0.201021,0.012635,0.010895,1.000000,0.123091,0.103675,0.121415,0.089324,0.010032,0.051755,0.068041,0.060141,0.085544,0.078567,0.076129,0.185577,0.152724,0.127547,0.192467,0.104973,0.071120,0.049899,0.111340,0.077850,0.040032,0.059391,0.128834,0.145347,0.068041,0.058345,0.115470,0.188639,0.174325,0.088045,0.070938,0.114193,0.059391,0.107858,0.231341,0.073747,...,0.012859,0.191283,0.133788,0.070938,0.056701,0.015215,0.155510,0.000000,0.119455,0.172705,0.075308,0.054554,0.054554,0.052280,0.095830,0.080203,0.169504,0.045462,0.075308,0.194177,0.138889,0.070179,0.221242,0.089343,0.136931,0.107399,0.258619,0.044544,0.174957,0.204451,0.101556,0.134690,0.307987,0.061237,0.073231,0.195617,0.144295,0.174664,0.033558,0.099941
5,0.128669,0.027995,0.024140,0.123091,1.000000,0.306275,0.110051,0.417810,0.000000,0.038224,0.188445,0.079950,0.027077,0.282876,0.103800,0.076143,0.147122,0.121114,0.096660,0.067837,0.057301,0.027639,0.068525,0.100618,0.029566,0.328976,0.090825,0.113660,0.083753,0.077563,0.063960,0.223906,0.253472,0.065026,0.345782,0.019462,0.361873,0.392604,0.150756,0.341651,...,0.014245,0.121090,0.139495,0.408651,0.025126,0.000000,0.130693,0.000000,0.158802,0.122738,0.047673,0.000000,0.100728,0.413690,0.019302,0.083624,0.152572,0.402911,0.214529,0.134097,0.000000,0.342084,0.178253,0.148464,0.067420,0.096671,0.121765,0.000000,0.090854,0.152816,0.060003,0.402224,0.122732,0.286436,0.141973,0.103840,0.165365,0.156890,0.247841,0.058492


### Calculate the preferences

In [ ]:
# 計算 Users 對 Items 的偏好程度
P = S@rating_pivot
print(P)

movieId     1          2          3       ...    193585    193587    193609
userId                                    ...                              
1        37.740124  20.266241  10.789043  ...  0.005672  0.005672  0.050653
2        11.167948   5.755950   1.493934  ...  0.096250  0.096250  0.171920
3         5.905909   3.384914   1.722441  ...  0.000000  0.000000  0.000000
4        26.923313  13.479705   6.525372  ...  0.011756  0.011756  0.062994
5        33.718201  20.023260   8.648215  ...  0.000000  0.000000  0.058155
...            ...        ...        ...  ...       ...       ...       ...
606      35.021499  19.181163   8.121257  ...  0.018110  0.018110  0.140941
607      37.208653  20.057047   9.312567  ...  0.006317  0.006317  0.045135
608      51.217277  31.066904  14.681630  ...  0.017980  0.017980  0.224814
609      28.468446  16.281960   7.406512  ...  0.000000  0.000000  0.038051
610      34.247052  19.408169   6.722207  ...  0.112523  0.112523  0.284375

[610 rows x

### Prediction for the user

In [ ]:
# 取出第5號用戶的分數
u = P.loc[5]
print(type(u))

# 排序
sorted_u = u.sort_values(ascending=False)
sorted_u

# 取前5名
top_ids = pd.DataFrame(sorted_u[:5].index)
top_ids

<class 'pandas.core.series.Series'>


,movieId
0,296
1,356
2,318
3,110
4,150


In [ ]:
# 取出電影名稱

#順序會被重排
#top_movies = movies[movies['movieId'].isin(top_index)]
#top_movies

top_movies = pd.merge(top_ids,movies,on='movieId')
top_movies.title

0                 Pulp Fiction (1994)
1                 Forrest Gump (1994)
2    Shawshank Redemption, The (1994)
3                   Braveheart (1995)
4                    Apollo 13 (1995)
Name: title, dtype: object

# The example for lecture

In [ ]:
import numpy as np
import pandas as pd

a = np.array([
    [3, 4, 0, 3.5, 0],
    [4, 0, 4.5, 0, 3.5],
    [0, 3.5, 0, 0, 3],
    [0, 4, 0, 3.5, 3]
])

#a = np.array([[3,0,2,0,5],[3,4,2,1,0],[0,0,0,0,3],[0,2,0,3,0]])
df = pd.DataFrame(data=a,index=['A','B','C','D'],columns=['a','b','c','d','e'])
df

,a,b,c,d,e
A,3.0,4.0,0.0,3.5,0.0
B,4.0,0.0,4.5,0.0,3.5
C,0.0,3.5,0.0,0.0,3.0
D,0.0,4.0,0.0,3.5,3.0


In [ ]:
# 轉成是否有評分
df[df > 0] = 1.0
df

,a,b,c,d,e
A,1.0,1.0,0.0,1.0,0.0
B,1.0,0.0,1.0,0.0,1.0
C,0.0,1.0,0.0,0.0,1.0
D,0.0,1.0,0.0,1.0,1.0


In [ ]:
A = df

A.T

,A,B,C,D
a,1.0,1.0,0.0,0.0
b,1.0,0.0,1.0,1.0
c,0.0,1.0,0.0,0.0
d,1.0,0.0,0.0,1.0
e,0.0,1.0,1.0,1.0


In [ ]:
#共現矩陣
#C = A.dot(A.T)
C = A@A.T
C

,A,B,C,D
A,3.0,1.0,1.0,2.0
B,1.0,3.0,1.0,1.0
C,1.0,1.0,2.0,2.0
D,2.0,1.0,2.0,3.0


In [ ]:
# 取出 C 的對角為每個用戶各自的次數
arr = C.to_numpy()
user_rating_counts = np.diagonal(arr)
user_rating_counts

array([3., 3., 2., 3.])

In [ ]:
arr2 = np.tile(user_rating_counts, (len(user_rating_counts),1))
X = pd.DataFrame(data=arr2.T)
X

,0,1,2,3
0,3.0,3.0,3.0,3.0
1,3.0,3.0,3.0,3.0
2,2.0,2.0,2.0,2.0
3,3.0,3.0,3.0,3.0


In [ ]:
X.T

,0,1,2,3
0,3.0,3.0,2.0,3.0
1,3.0,3.0,2.0,3.0
2,3.0,3.0,2.0,3.0
3,3.0,3.0,2.0,3.0


In [ ]:
# 分母部份：C' = N(u)*N(v)
C2 = X*X.T
C2.index = ['A','B','C','D']
C2.columns = ['A','B','C','D']
C2

,A,B,C,D
A,9.0,9.0,6.0,9.0
B,9.0,9.0,6.0,9.0
C,6.0,6.0,4.0,6.0
D,9.0,9.0,6.0,9.0


In [ ]:
# 分母部份：C'開根號
D = np.sqrt(C2)
D

,A,B,C,D
A,3.00000,3.00000,2.44949,3.00000
B,3.00000,3.00000,2.44949,3.00000
C,2.44949,2.44949,2.00000,2.44949
D,3.00000,3.00000,2.44949,3.00000


In [ ]:
#計算最後的相似度矩陣
SIM = C/D
SIM

,A,B,C,D
A,1.000000,0.333333,0.408248,0.666667
B,0.333333,1.000000,0.408248,0.333333
C,0.408248,0.408248,1.000000,0.816497
D,0.666667,0.333333,0.816497,1.000000
